In [1]:
import BioSimSpace as BSS
import glob

/home/nichrun/miniconda3/envs/biosimspace/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/nichrun/miniconda3/envs/biosimspace/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/nichrun/miniconda3/envs/biosimspace/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/nichrun/miniconda3/envs/biosimspace/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
#BSS
#ligands = [BSS.IO.readMolecules(x) for x in glob.glob("ClusterMolecules10/*.sdf")]

In [3]:
from rdkit import Chem

ligands = [Chem.SDMolSupplier(x)[0] for x in glob.glob("ClusterMolecules10/*.sdf")]
ligand_names = [x for x in glob.glob("ClusterMolecules10/*.sdf")]
ligand_names

['ClusterMolecules10/13655.sdf',
 'ClusterMolecules10/13659.sdf',
 'ClusterMolecules10/13384.sdf',
 'ClusterMolecules10/13385.sdf',
 'ClusterMolecules10/13209.sdf',
 'ClusterMolecules10/16722.sdf',
 'ClusterMolecules10/13664.sdf',
 'ClusterMolecules10/13654.sdf',
 'ClusterMolecules10/13210.sdf',
 'ClusterMolecules10/13662.sdf']

In [4]:
import csv
true_perts = []
true_sems = []

with open("cluster_pair_preds.csv", "r") as readfile:
    reader = csv.reader(readfile)
    next(reader)
    for row in reader:
        true_perts.append([row[0], row[1]])
        true_sems.append(row[-1])

In [5]:
with open("links_file.in", "w") as writefile:
    writer = csv.writer(writefile, delimiter =" ")
    for (a,b), value in zip(true_perts, true_sems):
        
        writer.writerow([a, b, value])

In [6]:
tranformations, lomap_scores = BSS.Align.generateNetwork(ligands, plot_network=True, names=ligand_names, 
                                                         work_dir="runner_work_dir",
                                                         links_file="links_file.in"
                                                        )

OSError: File error: Bad output file /home/nichrun/Documents/data_driven_fep_reliabilities/CLI/runner_work_dir/inputs/000_ClusterMolecules10/13655.sdf.sdf


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.

